In [1]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


In [11]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [5]:
class X2E_NetMSE(torch.nn.Module):
    def __init__(self):
        super(X2E_NetMSE, self).__init__()
        self.layer1 = torch.nn.Sequential(nn.Conv1d(512,450,1), nn.ReLU())
        self.layer2 = torch.nn.Sequential(nn.Conv1d(450,400,1), nn.ReLU())
        self.layer3 = torch.nn.Sequential(nn.Conv1d(400,350,1), nn.ReLU())
        self.layer4 = torch.nn.Sequential(nn.Conv1d(350,300,1), nn.ReLU())
        self.fc = torch.nn.Linear(300,256)
    
    def forward(self, x):
        x=x/torch.norm(x,p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x=x/torch.norm(x,p=2)
        return x


In [6]:
# 模型
model= X2E_NetMSE()
# criterion = torch.nn.MSELoss(reduce=True, size_average=False) reduction='sum'
criterion = torch.nn.MSELoss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# 训练模型
epoch = 0
totaldir=r'D:\pycharm_project\vs2ttsvoice\X2E'
#损失值写入文件

se_dir=r'D:\pycharm_project\vs2ttsvoice\SE_vector'
for i in range(80):
    print(i)
    pathstr ='xvector'+str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir,pathstr)
    sepath='dpdir'+str(i+1)+'.txt' 
    sepath=os.path.join(se_dir,sepath)
    se_vectors=open(sepath).readlines()
    file=open( pathstr)
    j=0
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        se=se_vectors[j].replace('[','').replace(']','').split()
        se_filename=se[0]
        filename=data[0]
        if se_filename==filename:
#             print(se_filename)
            data=data[1:]
            se_data=se[1:]
            out=[[list(map(float,data))]]
            out=torch.tensor(out)
            out=out.permute(0,2,1)
            pred=model(out)
            pred=pred.squeeze(0)
            label=list(map(float,se_data))
            label=torch.tensor(label)
            loss = criterion(pred, label)
            print_loss = loss.data.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch+=1
            if epoch%100 == 0:
            #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
            #保存参数、
                print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
                state = { 'model':model.state_dict(), 'epoch':epoch }                       
                if not os.path.isdir('checkpointX2E'):
                     os.mkdir('checkpointX2E')    
                torch.save(state, '.\checkpointX2E\modelconvmsetest_epoch_%d.ckpt' % (epoch))
                file_save = open('lossX2Econvmsetest.txt',mode='a')
                file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
                file_save.close() 
        else:continue
        j=j+1

0
epoch: 100, loss: 0.1721
epoch: 200, loss: 0.2704
epoch: 300, loss: 0.08164
epoch: 400, loss: 0.4353
epoch: 500, loss: 0.3485
epoch: 600, loss: 0.1813
epoch: 700, loss: 0.175
epoch: 800, loss: 0.2068
epoch: 900, loss: 0.2374
epoch: 1000, loss: 0.267
epoch: 1100, loss: 0.4741
epoch: 1200, loss: 0.3864
epoch: 1300, loss: 0.4167
epoch: 1400, loss: 0.4808
epoch: 1500, loss: 0.3304
epoch: 1600, loss: 0.124
epoch: 1700, loss: 0.707
epoch: 1800, loss: 0.2974
1
epoch: 1900, loss: 0.2492
epoch: 2000, loss: 0.3003
epoch: 2100, loss: 0.1506
epoch: 2200, loss: 0.6338
epoch: 2300, loss: 0.1218
epoch: 2400, loss: 0.2881
epoch: 2500, loss: 0.1453
epoch: 2600, loss: 0.2508
epoch: 2700, loss: 0.09577
epoch: 2800, loss: 0.586
epoch: 2900, loss: 0.09941
epoch: 3000, loss: 0.2772
epoch: 3100, loss: 0.1638
epoch: 3200, loss: 0.3174
epoch: 3300, loss: 0.6876
epoch: 3400, loss: 0.2716
epoch: 3500, loss: 0.4205
epoch: 3600, loss: 0.1852
epoch: 3700, loss: 0.4009
2
epoch: 3800, loss: 0.6131
epoch: 3900, loss

epoch: 30800, loss: 0.3711
17
epoch: 30900, loss: 0.2537
epoch: 31000, loss: 0.4054
epoch: 31100, loss: 0.3836
epoch: 31200, loss: 0.3939
epoch: 31300, loss: 0.4279
epoch: 31400, loss: 0.3784
epoch: 31500, loss: 0.2832
epoch: 31600, loss: 0.3347
epoch: 31700, loss: 0.4383
epoch: 31800, loss: 0.2876
epoch: 31900, loss: 0.4137
epoch: 32000, loss: 0.2858
epoch: 32100, loss: 0.4264
epoch: 32200, loss: 0.4268
epoch: 32300, loss: 0.3235
epoch: 32400, loss: 0.318
epoch: 32500, loss: 0.4974
epoch: 32600, loss: 0.3046
18
epoch: 32700, loss: 0.428
epoch: 32800, loss: 0.1684
epoch: 32900, loss: 0.2294
epoch: 33000, loss: 0.3886
epoch: 33100, loss: 0.193
epoch: 33200, loss: 0.3856
epoch: 33300, loss: 0.3833
epoch: 33400, loss: 0.3685
epoch: 33500, loss: 0.3069
epoch: 33600, loss: 0.3638
epoch: 33700, loss: 0.4261
epoch: 33800, loss: 0.3469
epoch: 33900, loss: 0.347
epoch: 34000, loss: 0.1819
epoch: 34100, loss: 0.1459
epoch: 34200, loss: 0.2381
epoch: 34300, loss: 0.3909
epoch: 34400, loss: 0.4487

epoch: 61100, loss: 0.4489
epoch: 61200, loss: 0.3111
epoch: 61300, loss: 0.3807
epoch: 61400, loss: 0.3278
epoch: 61500, loss: 0.3363
epoch: 61600, loss: 0.3221
epoch: 61700, loss: 0.3608
epoch: 61800, loss: 0.336
epoch: 61900, loss: 0.1943
epoch: 62000, loss: 0.2136
epoch: 62100, loss: 0.5276
epoch: 62200, loss: 0.1934
epoch: 62300, loss: 0.2652
34
epoch: 62400, loss: 0.2776
epoch: 62500, loss: 0.2771
epoch: 62600, loss: 0.2434
epoch: 62700, loss: 0.2541
epoch: 62800, loss: 0.1856
epoch: 62900, loss: 0.2341
epoch: 63000, loss: 0.4461
epoch: 63100, loss: 0.2437
epoch: 63200, loss: 0.2468
epoch: 63300, loss: 0.2472
epoch: 63400, loss: 0.3594
epoch: 63500, loss: 0.205
epoch: 63600, loss: 0.1763
epoch: 63700, loss: 0.2762
epoch: 63800, loss: 0.3432
epoch: 63900, loss: 0.162
epoch: 64000, loss: 0.4195
epoch: 64100, loss: 0.2156
35
epoch: 64200, loss: 0.3687
epoch: 64300, loss: 0.3505
epoch: 64400, loss: 0.2364
epoch: 64500, loss: 0.168
epoch: 64600, loss: 0.3462
epoch: 64700, loss: 0.2025

epoch: 91400, loss: 0.2279
epoch: 91500, loss: 0.431
epoch: 91600, loss: 0.3841
epoch: 91700, loss: 0.3011
epoch: 91800, loss: 0.2748
epoch: 91900, loss: 0.5197
epoch: 92000, loss: 0.1998
epoch: 92100, loss: 0.3779
51
epoch: 92200, loss: 0.3323
epoch: 92300, loss: 0.3791
epoch: 92400, loss: 0.2782
epoch: 92500, loss: 0.4463
epoch: 92600, loss: 0.5036
epoch: 92700, loss: 0.4205
epoch: 92800, loss: 0.2542
epoch: 92900, loss: 0.2881
epoch: 93000, loss: 0.3302
epoch: 93100, loss: 0.3485
epoch: 93200, loss: 0.2896
epoch: 93300, loss: 0.2617
epoch: 93400, loss: 0.2934
epoch: 93500, loss: 0.2319
epoch: 93600, loss: 0.2148
epoch: 93700, loss: 0.1826
epoch: 93800, loss: 0.2914
epoch: 93900, loss: 0.3258
52
epoch: 94000, loss: 0.2422
epoch: 94100, loss: 0.334
epoch: 94200, loss: 0.2784
epoch: 94300, loss: 0.3319
epoch: 94400, loss: 0.2365
epoch: 94500, loss: 0.3021
epoch: 94600, loss: 0.2612
epoch: 94700, loss: 0.2925
epoch: 94800, loss: 0.4277
epoch: 94900, loss: 0.3077
epoch: 95000, loss: 0.34

epoch: 121000, loss: 0.3535
epoch: 121100, loss: 0.1975
epoch: 121200, loss: 0.3571
epoch: 121300, loss: 0.4007
epoch: 121400, loss: 0.2621
epoch: 121500, loss: 0.2651
epoch: 121600, loss: 0.3069
epoch: 121700, loss: 0.2592
66
epoch: 121800, loss: 0.4758
epoch: 121900, loss: 0.283
epoch: 122000, loss: 0.2491
epoch: 122100, loss: 0.3343
epoch: 122200, loss: 0.3293
epoch: 122300, loss: 0.2216
epoch: 122400, loss: 0.1777
epoch: 122500, loss: 0.2221
epoch: 122600, loss: 0.2237
epoch: 122700, loss: 0.2213
epoch: 122800, loss: 0.2629
epoch: 122900, loss: 0.2119
epoch: 123000, loss: 0.2827
epoch: 123100, loss: 0.3473
epoch: 123200, loss: 0.164
epoch: 123300, loss: 0.2952
epoch: 123400, loss: 0.353
epoch: 123500, loss: 0.2739
67
epoch: 123600, loss: 0.2852
epoch: 123700, loss: 0.3553
epoch: 123800, loss: 0.2833
epoch: 123900, loss: 0.2897
epoch: 124000, loss: 0.2406
epoch: 124100, loss: 0.1973
epoch: 124200, loss: 0.3297
epoch: 124300, loss: 0.3234
epoch: 124400, loss: 0.4123
epoch: 124500, lo

In [8]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpointX2E\modelconvmsetest_epoch_148600.ckpt')
model.load_state_dict(checkpoint['model'])
# model.eval()

<All keys matched successfully>

In [12]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open(r'.\X2E\pv_xvector.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
    out=[[list(map(float,data))]]
    out=torch.tensor(out)
    out=out.permute(0,2,1)
    pred=model(out)
    pred=pred.squeeze(0)
    pred=pred.tolist()
#     print("pred",len(pred))
#     print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6062351862491422


id10271-1gtz-CUIygI-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.605920916885796


id10272-0qqQJVFKpbA-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.7065454693191962


id10273-0LbtndiXJC0-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6071155201074327


id10274-7rkVfW2yw0c-00001
Synthesizing 预测语音...


KeyboardInterrupt: 